## Purity Calculation

$$ A^{sig+bkg}=u A^{sig}+(1-u)A^{bkg}$$

"where u is the exclusive event purity in the Sig+bkg region"

"purity is calculated by performing a binned fit over the invariant mass range 0.4 - 1.7 GeV and taking the fraction of integrated singal events over the total in the Signal+Background region."

To calculate this purity, we need to fit the invariant hadron mass plot two a function F(M) which is made of two functions F(M)=s(M)+b(M) where s(M) is a gaussian (for example) meant to fit to the signal shape, and b(M) is a polynomial meant to fit to the background shape.


In [1]:
import ROOT
import array
from ROOT import RooRealVar, RooArgSet, RooDataSet, RooAbsPdf, RooFit, TMath, RooDataHist

In [2]:
pass2inputFiles = ["out/pippi0_fall2018_in_pass2/pippi0_fall2018_in_pass2.root","out/pippi0_spring2019_in_pass2/pippi0_spring2019_in_pass2.root"]
pass1inputFiles = ["out/pippi0_fall2018_in_pass1/pippi0_fall2018_in_pass1.root","out/pippi0_spring2019_in_pass1/pippi0_spring2019_in_pass1.root"]

In [3]:
chain1 = ROOT.TChain("pippi0")
chain2 = ROOT.TChain("pippi0")
for f in pass1inputFiles:
    chain1.Add(f)
for f in pass2inputFiles:
    chain2.Add(f)

In [4]:
print(chain1.GetEntries())
[b.GetName() for b in chain1.GetListOfBranches()]

27038046


['hel',
 'run',
 'x',
 'eps',
 'gamma',
 'x_true',
 'Q2',
 'Q2_true',
 'z',
 'pT',
 'xF',
 'xF1',
 'xF2',
 'phi',
 'Mx',
 'Mh',
 'z_true',
 'pT_true',
 'xF_true',
 'xF1_true',
 'xF2_true',
 'phi_true',
 'Mx_true',
 'Mh_true',
 'Mdiphoton',
 'Mdiphoton_true',
 'th',
 'th_true',
 'cth',
 'cth_true']

In [5]:
#create RooReal variables
Mh = RooRealVar("Mh", "Mh", 0.4, 1.7)
Mh.setRange("fullRange",0.4,1.7)

mu = RooRealVar("m_{0}", "mu", 0.8,0.6, 1)
sigl = RooRealVar("#sigma_{L}", "sig_L", 0.06,0.00001, 2)
sigh = RooRealVar("#sigma_{R}", "sig_R", 0.06,0.00001, 2)
alphal = RooRealVar("#alpha_{L}", "alphal", 1.57,0.00001,10)
alphah = RooRealVar("#alpha_{R}", "alphah", 0.9,0.00001,10)
nl = RooRealVar("n_{L}", "nl", 1.7,0.00001,10)
nh = RooRealVar("n_{R}", "nh", 5,0.00001,10)
p1 = RooRealVar("p1", "p1", 0,-5,5)
p2 = RooRealVar("p2", "p2", 0,-5,5)
p3 = RooRealVar("p3", "p3", 0,-5,5)
p4 = RooRealVar("p4", "p4", 0,-5,5)


[#1] INFO:Eval -- RooRealVar::setRange(Mh) new range named 'fullRange' created with bounds [0.4,1.7]


In [6]:
N_dcb = RooRealVar("N_{dcb}", "N_dcb", 100000,0,chain2.GetEntries())
N_bkg = RooRealVar("N_{bkg}", "N_bkg", 100000,0,chain2.GetEntries())

dcb = ROOT.RooCrystalBall("dcb","Right Crystal Ball", Mh,mu,sigl,sigh,alphal,nl,alphah,nh)

pars_pol = ROOT.RooArgList(p1,p2,p3)#,p4)
background = ROOT.RooChebychev("background", "Background", Mh, pars_pol)

# Combine signal and background
model_ext = ROOT.RooAddPdf("model_ext", "Signal + Background", ROOT.RooArgList(dcb, background), ROOT.RooArgList(N_dcb,N_bkg))


In [7]:

try:
    del c
except NameError:
    pass
c = ROOT.TCanvas()#"c","c",800,600)
c.SetTickx()
c.SetTicky()
c.SetGridx()
c.SetGridy()

h = ROOT.TH1F("h","Hadron Mass M_{h};M_{h} [GeV];Events",200,0.4,1.7)
chain2.Draw("Mh>>h","Mdiphoton<0.16 && 0.115<Mdiphoton && 0.85<Mx && Mx < 1.05 && Mh>0.4 &&Mh<1.7","goff")
maxbin = h.GetMaximumBin()
maxcount = h.GetBinContent(maxbin)
h.Scale(1/maxcount)

Mdiphoton = RooRealVar("Mdiphoton","Mdiphoton",0.115,0.16)
Mx = RooRealVar("Mx","Mx",0.85,1.05)
h_roo = RooDataSet("h_roo","h_roo",ROOT.RooArgSet(Mh),
                   RooFit.Import(chain2))#,RooFit.Cut("Mdiphoton<0.16 && 0.115<Mdiphoton && 0.85<Mx && Mx < 1.05 && Mh>0.4 &&Mh<1.7"))     
#h_roo = RooDataHist("h_roo", "h_roo",ROOT.RooArgList(Mh),h)
fit_results = model_ext.fitTo(h_roo,RooFit.Range("fullRange"),
                              RooFit.Save(),RooFit.PrintLevel(-1),RooFit.SumW2Error(True),Extended = True)


KeyboardInterrupt: Could not find "fitTo<RooLinkedList const&>" (set cppyy.set_debug() for C++ errors):
  RooFitResult* RooAbsPdf::fitTo(RooAbsData& data, const RooLinkedList& cmdArgs) =>
    KeyboardInterrupt: 

[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(h_roo) Skipping event #72 because Mh cannot accommodate the value 0.361707
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(h_roo) Skipping event #73 because Mh cannot accommodate the value 0.361707
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(h_roo) Skipping event #76 because Mh cannot accommodate the value 0.391763
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(h_roo) Skipping event #77 because Mh cannot accommodate the value 0.391763
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(h_roo) Skipping ...
[#0] WARNING:DataHandling -- RooTreeDataStore::loadValues(h_roo) Ignored 2909556 out-of-range events
[#1] INFO:Eval -- RooRealVar::setRange(Mh) new range named 'fit_nll_model_ext_h_roo' created with bounds [0.4,1.7]
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model_ext) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using CPU computation library compiled wi

Warning in <ROOT::Math::Fitter::CalculateHessErrors>: Error when calculating Hessian
Warning in <ROOT::Math::Fitter::CalculateHessErrors>: Error when calculating Hessian


In [ ]:
fit_results.Print("v")
try:
    del frame
except NameError:
    pass
frame = Mh.frame(0.4,1.7)
frame.SetYTitle("Events")
h_roo.plotOn(frame,RooFit.MarkerSize(0.5),
            RooFit.Name("data"))
model_ext.plotOn(frame,RooFit.LineStyle(ROOT.kDashed),RooFit.LineColor(ROOT.kBlack),
            RooFit.Name("totalFit"))
model_ext.plotOn(frame, RooFit.Components("background"), RooFit.LineColor(ROOT.kRed),
            RooFit.Name("chebychevFit"))
model_ext.plotOn(frame, RooFit.Components("dcb"), RooFit.LineColor(ROOT.kBlue),
            RooFit.Name("dcbFit"))
frame.SetTitle("Purity Calculation Fit")

try:
    del canvas
except NameError:
    pass
canvas = ROOT.TCanvas()
canvas.Divide(2,1)

try:
    del leg
except NameError:
    pass
leg = ROOT.TLegend(0.55,0.55,0.85,0.85) #xmin,ymin,xmax.ymax
leg.AddEntry(frame.findObject("data"),"Data","p") #name of object being referred, "title shown", (fit line or point)
leg.AddEntry(frame.findObject("totalFit"),"dcb+chebychev fit","l")
leg.AddEntry(frame.findObject("dcbFit"),"dcb function","l")
leg.AddEntry(frame.findObject("chebychevFit"),"ChebyChev function","l")
leg.SetBorderSize(0)

pad1 = canvas.cd(1)
pad1.SetPad(0.0,0.1,0.9,1.0)
pad1 = canvas.cd(2)
pad1.SetPad(0.815,0.0,1.0,1.0)

u,u_err = integrate_u(Mh,background,dcb,N_dcb,N_bkg,fit_results)
text=ROOT.TLatex(0.525,0.45,f"u = {round(u,4)}\pm{round(u_err,4)}")
text.SetNDC(True)
text.SetTextSize(0.06)
text.SetTextColor(ROOT.kBlack)

canvas.cd(1)
frame.Draw()
leg.Draw()
text.Draw()

canvas.cd(2)
try:
    del param_box
except NameError:
    pass
param_box = ROOT.TPaveText(0.05, 0.2, 0.95, 0.8, "NDC")
param_box.SetFillColor(0)
param_box.SetTextAlign(22)
param_box.SetTextSize(0.1)
for p in model_ext.getParameters(h_roo):
    name = p.GetName()
    val = p.getVal()
    val_err = p.getError()
    if name=="N_{bkg}" or name== "N_{dcb}":
        param_box.AddText(f"{name}: {val:.0e}#pm{val_err:.0e}")
    else:
        param_box.AddText(f"{name}: {val:.2f}#pm{val_err:.2f}")

param_box.Draw()

canvas.Draw()


In [ ]:
def integrate_u(Mh,background,dcb,N_dcb,N_bkg,fit_results):
    #calculate u based on the integral of sig/sig+bkg in the sig+bkg region
    Mh.setRange("sigbkgRange",0.65,0.9)
    
    #the integral finds the amount of the function that is in the sigbkg region
    bkg_int = background.createIntegral(RooArgSet(Mh),RooArgSet(Mh),"sigbkgRange")
    bkg_perc = bkg_int.getVal()
    
    sig_int = dcb.createIntegral(RooArgSet(Mh),RooArgSet(Mh),"sigbkgRange")
    sig_perc = sig_int.getVal()
    
    #then multiply this percentage by the Number of sig(bkg) events found in the extended PDF fitting to the peak to get the 
    #total number of sig(bkg) events IN THAT REGION SPECIFICALLY
    sig_N_local = sig_perc*N_dcb.getVal()
    bkg_N_local = bkg_perc*N_bkg.getVal()
    
    #then divide to get the purity
    denom = (bkg_N_local + sig_N_local)
    num = sig_N_local
    u = num/denom  #integral of sig divided by integral of sig+bkg
    
    #calculate error
    bkg_perc_err = bkg_int.getPropagatedError(fit_results,Mh)
    sig_perc_err = sig_int.getPropagatedError(fit_results,Mh)
    num_err = (sig_perc_err/sig_perc + N_dcb.getError()/N_dcb.getVal())*num
    denom_err = num_err + (bkg_perc_err/bkg_perc + N_bkg.getError()/N_bkg.getVal())*bkg_N_local
    u_err = u*ROOT.TMath.Sqrt(ROOT.TMath.Power(num_err/num,2)+ROOT.TMath.Power(denom_err/denom,2))
    print(f"bkg_N: {round(bkg_N_local,3)} +/- {round(bkg_N_err,3)}, sig_N: {round(sig_N_local,3)} +/- {round(sig_N_err,3)}")
    print(f"{u} +/- {u_err}")
    return u,u_err

u,u_err = integrate_u(Mh,background,dcb,N_dcb,N_bkg,fit_results)

In [ ]:
# Create a TPaveText box
equation_box = ROOT.TPaveText(0.5, 0.5, 0.9, 0.9, "NDC")  # (x1, y1, x2, y2) in normalized device coordinates
equation_box.SetFillColor(0)
equation_box.SetTextFont(42)
equation_box.SetTextSize(0.025)
equation_box.SetBorderSize(1)

# Add the equation lines (simplified for readability)
equation_box.AddText("f(m; m_{0}, #sigma_{L}, #alpha_{L}, n_{L}, #sigma_{R}, #alpha_{R}, n_{R}) =")
equation_box.AddText("  A_{L}(B_{L} - (m - m_{0})/#sigma_{L})^{-n_{L}}, if (m - m_{0})/#sigma_{L} < -#alpha_{L}")
equation_box.AddText("  exp(-0.5 * ((m - m_{0})/#sigma_{L})^{2}), if -#alpha_{L} ≤ 0")
equation_box.AddText("  exp(-0.5 * ((m - m_{0})/#sigma_{R})^{2}), if 0 ≤ #alpha_{R}")
equation_box.AddText("  A_{R}(B_{R} + (m - m_{0})/#sigma_{R})^{-n_{R}}, otherwise")
equation_box.AddText("A_{i} = (n_{i}/|#alpha_{i}|)^{n_{i}} * exp(-|#alpha_{i}|^{2}/2)")
equation_box.AddText("B_{i} = (n_{i}/|#alpha_{i}|) - |#alpha_{i}|")

# Draw the box on the canvas
#equation_box.Draw()

In [ ]:
for p in background_ext.getParameters(h_roo):
    print(f"dcb,{p.GetName()} = {p.getVal()}")
print("\n")
for p in model_ext.getParameters(h_roo):
    print(f"model,{p.GetName()} = {p.getVal()}")
    

#print(model_ext.getParameters(h_roo).find("N_model").getVal())

In [ ]:
c1 = ROOT.TCanvas()
frames = []
pdf_list = model.pdfList()
for idx in range(pdf_list.getSize()):
    print(pdf_list.at(idx))
    

c1.Draw()
